In [1]:
%run Training_Helpers.ipynb

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def set_plot_size_cm(fig, w, h):
    fig.set_size_inches(w / 2.54, h / 2.54)

def plot_training_metric(plot, trainingResult, metric, label):
    stepCount = trainingResult.train_accuracy.shape[0]
    steps = range(0, stepCount)

    plot.set_ylabel(label)
    plot.set_xlabel('Step')
    plot.plot(steps, trainingResult.train_accuracy[metric])
    plot.plot(steps, trainingResult.test_accuracy[metric])
    plot.set_xlim([0, stepCount])

def plot_training_metrics(trainingResult):
    fig, (pltLoss, pltAcc) = plt.subplots(1, 2)
    set_plot_size_cm(fig, 30, 5)

    plot_training_metric(pltLoss, trainingResult, 'loss', 'Loss')
    pltLoss.set_yscale('log')
    plot_training_metric(pltAcc, trainingResult, 'acc', 'Accuracy')
    pltAcc.set_ylim([0, 1])
    plt.show()

In [3]:
def train_neural_network(
    make_model_fn,
    train_data,
    test_data,
    input_fn,
    output_fn,

    max_epochs = 50000,
    check_epochs = 10,
    improvement_ratio_required = 1e-3,
    consecutive_passes_required = 100,

    plot_metrics = True,
):
    train_in = input_fn(train_data)
    train_out = output_fn(train_data)
    test_in = input_fn(test_data)
    test_out = output_fn(test_data)

    model = make_model_fn(train_in, train_out)

    train_accuracy = []
    test_accuracy = []
    old_train_loss = 1e10
    checks = 0
    for epoch in range(0, max_epochs):
        train_result = model.train_on_batch(train_in, train_out)
        train_accuracy.append(train_result)
        test_accuracy.append(model.evaluate(test_in, test_out, verbose = 0))
        if epoch % check_epochs == 0:
            train_loss = train_result[0]
            if train_loss > old_train_loss:
                # got worse; do not advance or reset checks
                continue
            if old_train_loss / train_loss - 1 > improvement_ratio_required:
                # improved significantly; reset checks and update reference
                old_train_loss = train_loss
                checks = 0
                continue
            checks += 1
            if checks >= consecutive_passes_required:
                break

    train_accuracy = pd.DataFrame(columns=model.metrics_names, data=train_accuracy)
    test_accuracy = pd.DataFrame(columns=model.metrics_names, data=test_accuracy)

    print('  train: loss = {:.8}  accuracy = {:.4}    test: loss = {:.8}  accuracy = {:.4}'.format(
        train_accuracy.iloc[-1][0],
        train_accuracy.iloc[-1]['acc'],
        test_accuracy.iloc[-1][0],
        test_accuracy.iloc[-1]['acc'],
    ))

    result = TrainingResult(model, train_accuracy, test_accuracy)
    if plot_metrics:
        plot_training_metrics(result)
    return result

In [4]:
def train_neural_networks(
    make_model_fn,
    train_data,
    test_data,
    input_fns,
    output_fn,
    **trainArgs
):
    return train_models(
        train_neural_network,
        make_model_fn,
        train_data,
        test_data,
        input_fns,
        output_fn,
        **trainArgs
    )